extract pdf with pdfminer

In [ ]:
pip install pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.4 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140082 sha256=90237f7415c629727e266294810c737874f9512619d1985c2be6f114c6b4f180
  Stored in directory: /root/.cache/pip/wheels/56/24/93/05316c6df89ff210a9a705060277e3acbfd2d1bd3a5853ee19
Successfully built pdfminer


In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import os
import shutil

uploaded = files.upload()

# Buat folder jika belum ada
os.makedirs("data/pdf", exist_ok=True)

# Pindahkan file ke folder data/pdf
for filename in uploaded.keys():
    shutil.move(filename, f"data/pdf/{filename}")


Saving putusan_2_pid.sus-tpk_2025_pn_sby_20250620125858.pdf to putusan_2_pid.sus-tpk_2025_pn_sby_20250620125858.pdf
Saving putusan_3_pid.sus-tpk_2025_pn_sby_20250620125818.pdf to putusan_3_pid.sus-tpk_2025_pn_sby_20250620125818.pdf
Saving putusan_5_pid.sus-tpk_2025_pn_sby_20250620125735.pdf to putusan_5_pid.sus-tpk_2025_pn_sby_20250620125735.pdf
Saving putusan_7_pid.sus-tpk_2025_pn_sby_20250620125307.pdf to putusan_7_pid.sus-tpk_2025_pn_sby_20250620125307.pdf
Saving putusan_8_pid.sus-tpk_2025_pn_sby_20250618143419.pdf to putusan_8_pid.sus-tpk_2025_pn_sby_20250618143419.pdf
Saving putusan_9_pid.sus-tpk_2025_pn_sby_20250618142838.pdf to putusan_9_pid.sus-tpk_2025_pn_sby_20250618142838.pdf
Saving putusan_12_pid.sus-tpk_2025_pn_sby_20250620125638.pdf to putusan_12_pid.sus-tpk_2025_pn_sby_20250620125638.pdf
Saving putusan_14_pid.sus-tpk_2025_pn_sby_20250620125409.pdf to putusan_14_pid.sus-tpk_2025_pn_sby_20250620125409.pdf
Saving putusan_15_pid.sus-tpk_2025_pn_sby_20250620125447.pdf to putu

Convert & Extraction Text

In [ ]:
from pdfminer.high_level import extract_text
import os
import re

# Lokasi folder
input_folder = 'data/pdf/'
output_folder = 'data/convert/'  # jangan pakai '/' di depan kalau bukan root
os.makedirs(output_folder, exist_ok=True)

# Ambil semua file PDF
pdf_files = [f for f in os.listdir(input_folder) if f.endswith(".pdf")]

for idx, filename in enumerate(pdf_files, 1):
    pdf_path = os.path.join(input_folder, filename)
    text = extract_text(pdf_path)

    # 1. Bersihkan dan gabungkan setiap baris dengan spasi
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    combined = ' '.join(lines)

    # 2. Sisipkan spasi di antara huruf kecil dan huruf kapital yang menempel (contoh: "JiminTempat")
    combined = re.sub(r'([a-z])([A-Z])', r'\1 \2', combined)

    # 3. Hilangkan spasi ganda akibat proses join
    cleaned_text = re.sub(r'\s{2,}', ' ', combined)

    # Simpan hasil ekstraksi ke file .txt dengan nama putusan001.txt, dst.
    output_path = os.path.join(output_folder, f"putusan{idx:03}.txt")
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(cleaned_text)

Preprocessing

In [ ]:
def clean_header_footer(text):
    lines = text.splitlines()  # Pisah jadi list per baris
    cleaned_lines = []

    for line in lines:
        # Hilangkan kata-kata target dari tiap baris
        line = line.replace("Direktori Putusan Mahkamah Agung Republik Indonesia", "")
        line = line.replace("Mahkamah Agung Republik Indonesia", "")
        line = line.replace("putusan.mahkamahagung.go.id", "")

        # Hapus baris jika kosong setelah dibersihkan
        if line.strip():  # cek jika masih ada isi
            cleaned_lines.append(line.strip())

    return '\n'.join(cleaned_lines)

In [ ]:
import re

def clean_footer(text):
    cleaned_text = re.sub(
        r"Disclaimer[\s\S]*?(Email\s*:\s*.+?[\n\r]+)?Telp\s*:\s*\d{3}[-\s]?\d{3,4}(?:[-\s]?\d{3,4})?\s*\(ext[.]?\s*\d+\)",
        "",
        text,
        flags=re.IGNORECASE
    )
    return cleaned_text.strip()

In [ ]:
import re

def clean_putusan_text(text):
    # Bersihkan semua variasi "Halaman X ..." + Putusan Nomor ...
    patterns = [
        r'(?i)halaman\s+\d+\s*/\s*\d+\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)halaman\s+\d+\s*/\s*\d+[,]?\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)halaman\s+\d+\s+dari\s+\d+\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)halaman\s+\d+\s+dari\s+\d+\s+halaman\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)hal\s+\d+\s+dari\s+\d+\s+hal\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)hal\s+\d+\s+dari\s+\d+\s+hal[.]?\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)halaman\s+\d+\s+putusan\s+nomor\s+.*?Sby',
        r'(?i)halaman\s+\d+\s+dari\s+\d+\s+halaman\s+putusan\s+nomor\s*:?\s+.*?Sby',
        r'(?i)hal\s+\d+\s+dari\s+\d+\s+hal\s+putusan\s+nomor\s+.*?Sby',
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Gabungkan "P U T U S A N" menjadi "PUTUSAN"
    text = re.sub(r"P\s+U\s+T\s+U\s+S\s+A\s+N", "PUTUSAN ", text, flags=re.IGNORECASE)

    # Bersihkan spasi ganda & newline berlebihan
    text = re.sub(r'[ \t]{2,}', ' ', text)
    text = re.sub(r'\n{2,}', '\n', text)

    return text.strip()


In [ ]:
import re
def normalize_text(text):
    text = text.lower()                             # lowercase semua
    text = re.sub(r'\b(?:[a-zA-Z]\s+){2,}[a-zA-Z]\b', lambda m: m.group(0).replace(' ', ''), text)

    # Ubah semua variasi tgl lahir → tanggal lahir
    text = re.sub(r"(?i)\btgl[.\s_]*lahir\b", "tanggal lahir", text)

    # Ganti hanya 'nama' yang berdiri sendiri, tanpa mengubah 'nama lengkap'
    text = re.sub(r"(?i)\bnama\b(?!\s+lengkap)", "nama lengkap", text)


    return text

In [ ]:
def clean_text_pipeline(text):
    text = clean_header_footer(text)
    text = clean_footer(text)
    text = clean_putusan_text(text)

    # Hapus semua "Halaman X"
    text = re.sub(r"(?i)\bhalaman\s*\d+\b", "", text)

    text = normalize_text(text)
    return text

In [ ]:
import os

input_path = "data/convert/putusan_putusan_86_pid.sus-tpk_2024_pn_sby_20250618152925.txt"
output_path = "data/raw/hasil1.txt"

# Pastikan folder output ada
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Baca isi file
with open(input_path, "r", encoding="utf-8") as f:
    text = f.read()

# Bersihkan teks
cleaned_text = clean_text_pipeline(text)

# Simpan hasilnya
with open(output_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

In [ ]:
import os

input_folder = "data/convert"
output_folder = "data/raw"

# Pastikan folder output ada
os.makedirs(output_folder, exist_ok=True)

# Ambil semua file .txt dari folder input dan urutkan
txt_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".txt")])

for i, filename in enumerate(txt_files, start=1):
    input_path = os.path.join(input_folder, filename)

    with open(input_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Bersihkan teks dengan pipeline
    cleaned_text = clean_text_pipeline(text)

    # Format nama file output: case_001.txt, case_002.txt, ...
    output_filename = f"case_{i:03}.txt"
    output_path = os.path.join(output_folder, output_filename)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(cleaned_text)